In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [3]:
import pandas as pd

In [4]:
collected, questions =  read_urls_questions(os.path.join('data/amsterdam/amsterdam_full.csv'),
                                            os.path.join('data/question_answer/questions.csv')) # read collected urls and questions + remove unsuccessful collection

In [5]:
len(collected)

8908

In [6]:
len(questions)

19134

In [7]:
es_client = Elasticsearch("http://localhost:9200")

In [17]:
question_list = get_questions(questions)
document_list = get_url_content_tuples(collected)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [18]:
es_client = set_index(es_client=es_client, collected=collected, mappings=mappings) # 1 min

In [19]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=5) # 15 gives the best results ; when i set it to 5 gives 0,26

In [ ]:
import pickle
with open ('data/results/bm25_ALLquestions_ALLamsterdam_docs.pickle', 'wb') as f:
    pickle.dump(results, f)

In [9]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources for Dutch tokenization
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [10]:
# Create an empty dataframe for the new data
new_df = pd.DataFrame(columns=['URL', 'Textual_Content'])

# Iterate over each row in the original dataframe
for index, row in collected.iterrows():
    url = row['URL']
    content = str(row['Textual_Content'])
    
    # Tokenize the content into words
    words = word_tokenize(content, language='dutch')
    
    # Create passages of 100 words
    passages = [' '.join(words[i:i+100]) for i in range(0, len(words), 100)]
    
    # Create a new dataframe for the passages
    passage_df = pd.DataFrame({'URL': url, 'Textual_Content': passages})
    
    # Concatenate the new dataframe with the main dataframe
    new_df = pd.concat([new_df, passage_df], ignore_index=True)


In [11]:
document_list = get_url_content_tuples(new_df)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [12]:
es_client = set_index(es_client=es_client, collected=new_df, mappings=mappings) # 5 min

In [13]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=10) # 10 min

In [15]:
import pickle

In [16]:
with open ('data/results/bm25_ALLquestions_ALLamsterdam_pars.pickle', 'wb') as f:
    pickle.dump(results, f)